In [1]:
# Import the InferencePipeline object
from inference import InferencePipeline
# Import the built in render_boxes sink for visualizing results
from inference.core.interfaces.stream.sinks import render_boxes

# initialize a pipeline object
pipeline = InferencePipeline.init(
    model_id="rock-paper-scissors-sxsw/11", # Roboflow model to use
    video_reference=0, # Path to video, device id (int, usually 0 for built in webcams), or RTSP stream url
    on_prediction=render_boxes, # Function to run after each prediction
)
# pipeline.start()
# pipeline.join()

[09/10/24 15:49:01] WARNING  Your inference package version 0.17.1 is out of date! Please upgrade to __init__.py:41
                             version 0.18.1 of inference for the latest features and bug fixes by                  
                             running `pip install --upgrade inference`.                                            

SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.


RoboflowAPINotAuthorizedError: Unauthorized access to roboflow API - check API key. Visit https://docs.roboflow.com/api-reference/authentication#retrieve-an-api-key to learn how to retrieve one.

In [5]:
# import the InferencePipeline interface
from inference import InferencePipeline
# import a built-in sink called render_boxes (sinks are the logic that happens after inference)
from inference.core.interfaces.stream.sinks import render_boxes

pipeline = InferencePipeline.init_with_yolo_world(
    video_reference=r"videos\vehicle-counting.mp4",
    classes=["car"],
    model_size="s",
    on_prediction=render_boxes,
)
# start the pipeline
pipeline.start()
# wait for the pipeline to finish
pipeline.join()

Creating inference sessions


CLIP model loaded in 6.73 seconds


: 

In [2]:
import cv2

cap = cv2.VideoCapture(r"videos\vehicle-counting.mp4")

# Get the video's frame width and height
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

# release the VideoCapture object
cap.release()

In [ ]:
import numpy as np
import supervision as sv
from ultralytics import YOLO
import cv2

model = YOLO(r"models\yolov10n.pt")
tracker = sv.ByteTrack()
box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()

# Calculate aspect ratio
aspect_ratio = width / height

# Create a named window with the ability to resize
cv2.namedWindow('frame', cv2.WINDOW_NORMAL)

# Resize the window to the desired size (e.g., 800x600)
cv2.resizeWindow('frame', int(600 * aspect_ratio), 600)

def callback(frame: np.ndarray, _: int) -> np.ndarray:
    results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)
    detections = tracker.update_with_detections(detections)

    labels = [
        f"#{tracker_id} {results.names[class_id]}"
        for class_id, tracker_id
        in zip(detections.class_id, detections.tracker_id)
    ]

    annotated_frame = box_annotator.annotate(
        frame.copy(), detections=detections)
    label_annotator.annotate(
        annotated_frame, detections=detections, labels=labels)
    
    # display the frame to the user using OpenCV
    cv2.imshow("frame", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        return None
    
    return frame

sv.process_video(
    source_path=r"videos\vehicle-counting.mp4",
    target_path="result_111.mp4",
    callback=callback
)

In [3]:
import numpy as np
import supervision as sv
from ultralytics import YOLO
import cv2

model = YOLO(r"models\yolov10m.engine")
tracker = sv.ByteTrack()
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

# Calculate aspect ratio
aspect_ratio = width / height

# Create a named window with the ability to resize
cv2.namedWindow('frame', cv2.WINDOW_NORMAL)

# Resize the window to the desired size (e.g., 800x600)
cv2.resizeWindow('frame', int(600 * aspect_ratio), 600)

def callback(frame: np.ndarray, _: int) -> np.ndarray:
    results = model(frame, device = 'cuda')[0]
    detections = sv.Detections.from_ultralytics(results)
    detections = tracker.update_with_detections(detections)

    labels = [
        f"#{tracker_id} {results.names[class_id]}"
        for class_id, tracker_id
        in zip(detections.class_id, detections.tracker_id)
    ]

    annotated_frame = box_annotator.annotate(
        frame.copy(), detections=detections)
    label_annotator.annotate(
        annotated_frame, detections=detections, labels=labels)
    
    # display the frame to the user using OpenCV
    cv2.imshow("frame", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        return None
    
    return frame

sv.process_video(
    source_path=r"videos\vehicle-counting.mp4",
    target_path="result_111.mp4",
    callback=callback
)

WARNING  Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
None
None


SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.


Loading models\yolov10m.engine for TensorRT inference...



: 